# Reporte de Preparación de Datos para Fine-tuning con OpenAI

**Fecha:** 7 de junio de 2025

**Nombre:** Nelson Zepeda

**Correo Electrónico:** nelson.zepeda@datasphere.tech

**Dataset:** https://huggingface.co/datasets/Recognai/sentiment-banking/viewer/default/train
---

### Descripción del Archivo Generado

Este documento detalla el proceso de preparación de un dataset para el workshop de Fine-tuning con OpenAI. Específicamente, se ha tomado un archivo en formato Parquet alojado en Hugging Face (que contiene datos de `inputs` y `prediction` de un modelo de análisis de sentimiento bancario) y se ha transformado a un archivo JSONL (`.jsonl`). Este formato JSONL es el requerido por OpenAI para el fine-tuning de sus modelos de lenguaje, adaptándolos a una tarea de clasificación de texto.

---

### Marco Teórico

#### ¿Qué es Hugging Face?

**Hugging Face** es una empresa y una plataforma que se ha convertido en un centro neurálgico para el desarrollo y la implementación de modelos de Machine Learning, especialmente en el campo del Procesamiento de Lenguaje Natural (NLP) y la Visión por Computadora. Sus componentes más destacados incluyen:

* **Transformers Library:** Una librería de código abierto que proporciona arquitecturas de modelos de transformadores (como BERT, GPT, T5) pre-entrenados y listos para usar, facilitando tareas como la clasificación de texto, la generación de texto, la traducción y la respuesta a preguntas.
* **Hugging Face Hub:** Una plataforma centralizada que aloja miles de modelos, datasets y demos pre-entrenados por la comunidad y empresas. Permite compartir, descubrir y reutilizar recursos de ML de manera sencilla. Los datasets y modelos se pueden acceder directamente a través de URLs o APIs, como se vio en el uso del archivo Parquet.
* **Datasets Library:** Una librería que simplifica la descarga y el preprocesamiento de datasets para tareas de ML, optimizando el manejo de grandes volúmenes de datos.

Hugging Face promueve la investigación abierta y la democratización del acceso a las herramientas de IA, lo que la convierte en un recurso invaluable para desarrolladores y científicos de datos.

#### ¿Qué es Parquet?

**Parquet** es un formato de almacenamiento de datos columnar de código abierto, diseñado para un almacenamiento y procesamiento de datos eficiente, especialmente en el contexto de grandes volúmenes de datos (Big Data) y sistemas de procesamiento analítico. Sus características clave incluyen:

* **Almacenamiento Columnar:** A diferencia de los formatos de fila (como CSV), Parquet almacena los datos columna por columna. Esto es altamente eficiente para cargas de trabajo analíticas donde a menudo solo se necesita acceder a un subconjunto de columnas, ya que se pueden leer solo las columnas relevantes sin cargar toda la fila en memoria.
* **Compresión y Codificación Eficientes:** Ofrece altos ratios de compresión y varias técnicas de codificación que reducen el tamaño del archivo en disco, lo que se traduce en menos I/O (entrada/salida) y un procesamiento más rápido.
* **Esquema de Datos (Schema Evolution):** Soporta esquemas complejos y la evolución de esquemas, lo que significa que el esquema de los datos puede cambiar con el tiempo sin romper los datos existentes.
* **Soporte Multi-lenguaje:** Compatible con múltiples lenguajes y herramientas, incluyendo Python (Pandas, PyArrow), Java (Spark), R, etc.

Por su eficiencia en almacenamiento y velocidad de consulta, Parquet es un formato popular para lagos de datos y sistemas de procesamiento de datos distribuidos.

---

### Descripción del Script Python para la Extracción y Formateo

El script Python desarrollado tiene como objetivo principal transformar un dataset de análisis de sentimiento bancario almacenado en formato Parquet a un archivo JSONL (`.jsonl`) compatible con los requisitos de fine-tuning de OpenAI para modelos de chat.

**Funcionalidad del Script:**

1.  **Carga del Dataset Parquet:**
    * Utiliza la librería `pandas` y `huggingface_hub` (necesaria para el protocolo `hf://`) para leer directamente el archivo Parquet desde su ubicación en el Hugging Face Hub.
    * `df = pd.read_parquet("hf://datasets/Recognai/sentiment-banking/data/train-00000-of-00001.parquet")`

2.  **Selección y Limpieza de Columnas Relevantes:**
    * Se identifican las columnas `inputs` (que contiene el texto original del usuario) y `prediction` (que contiene la etiqueta de sentimiento predicha por un modelo base).
    * Se eliminan las filas que tienen valores nulos en cualquiera de estas dos columnas, asegurando que cada entrada de entrenamiento sea completa.

3.  **Procesamiento de la Columna `inputs`:**
    * La columna `inputs` inicialmente contenía el texto en un formato de objeto (e.g., `{"text": "Mi tarjeta dejó de funcionar..."}`).
    * Se implementa la función `extract_text_from_inputs` para extraer solo la cadena de texto pura de este objeto, ya que OpenAI espera contenido de usuario como una cadena simple. Se manejan casos donde la entrada podría no ser un diccionario o un JSON válido.

4.  **Procesamiento de la Columna `prediction`:**
    * La columna `prediction` contenía una cadena que representaba una lista de diccionarios con la etiqueta y el score (e.g., `[{'label': 'NEGATIVE', 'score': 0.999...}, {'label': 'POSITIVE', 'score': 0.000...}]`).
    * Se implementa la función `get_dominant_label` para:
        * Determinar el tipo de dato de la entrada (`str`, `list`, `numpy.ndarray`).
        * Si es una cadena, usa `ast.literal_eval` para convertirla de forma segura a una lista de diccionarios.
        * Si ya es una lista, la utiliza directamente.
        * Identifica la etiqueta (`label`) dentro de la lista de diccionarios que tiene el `score` más alto.
        * Maneja robustamente errores de formato o valores inesperados, devolviendo "UNKNOWN" si no puede extraer una etiqueta válida. Esto asegura que el `content` del rol `assistant` sea una única etiqueta de clase (`"POSITIVE"` o `"NEGATIVE"`).

5.  **Formato de Mensajes de OpenAI:**
    * Se define la función `create_openai_message` que toma las columnas procesadas (`processed_text` y `processed_label`) y las estructura en el formato de mensajes de chat que OpenAI requiere:
        ```json
        {"messages": [
            {"role": "user", "content": "El texto del usuario"},
            {"role": "assistant", "content": "La etiqueta de sentimiento"}
        ]}
        ```

6.  **Generación y Guardado del Archivo JSONL:**
    * La función `create_openai_message` se aplica a cada fila del DataFrame procesado para construir una lista de diccionarios en el formato de OpenAI.
    * Finalmente, esta lista se escribe en un archivo JSONL (`banking_sentiment_finetune_final.jsonl`), donde cada objeto JSON ocupa una línea separada.

Este script es un paso crucial para transformar datos tabulares o semi-estructurados en un formato utilizable para el fine-tuning de modelos de lenguaje, permitiendo que el modelo aprenda a clasificar textos bancarios basándose en los ejemplos proporcionados.

In [11]:
import pandas as pd
import json
import ast 
import numpy as np 

# Ruta al archivo parquet en Hugging Face
parquet_file_path = "hf://datasets/Recognai/sentiment-banking/data/train-00000-of-00001.parquet"

try:
    # Cargar el dataset Parquet
    df = pd.read_parquet(parquet_file_path)

    # Nombres de las columnas que nos interesan
    text_column_name = 'inputs'
    label_column_name = 'prediction'

    # Asegurarse de que las columnas existen y no tienen valores nulos críticos
    if text_column_name not in df.columns or label_column_name not in df.columns:
        raise ValueError(f"Las columnas '{text_column_name}' o '{label_column_name}' no se encontraron en el DataFrame.")

    # Filtrar filas donde 'inputs' o 'prediction' puedan ser nulos
    df_cleaned = df.dropna(subset=[text_column_name, label_column_name]).copy() # Usar .copy() para evitar SettingWithCopyWarning


    # Procesar la columna 'inputs' para extraer solo el texto
    def extract_text_from_inputs(input_obj):
        try:
            # Si ya es un diccionario, accede a 'text'
            if isinstance(input_obj, dict) and 'text' in input_obj:
                return input_obj['text']
            # Si es una cadena que parece JSON, parsea y luego accede
            elif isinstance(input_obj, str):
                parsed = json.loads(input_obj)
                if isinstance(parsed, dict) and 'text' in parsed:
                    return parsed['text']
            # Si no es un dict ni una cadena parseable con 'text', devuelve el original como string
            return str(input_obj)
        except (json.JSONDecodeError, KeyError):
            return str(input_obj) # Fallback si no se puede parsear o no tiene 'text'
    
    df_cleaned['processed_text'] = df_cleaned[text_column_name].apply(extract_text_from_inputs)


    # --- CORRECCIÓN FINAL AQUÍ: Procesar la columna 'prediction' para extraer solo la etiqueta ---
    def get_dominant_label(prediction_data):
        # Caso 1: prediction_data ya es una lista de diccionarios (Pandas ya lo parseó)
        if isinstance(prediction_data, list):
            pred_list = prediction_data
        # Caso 2: prediction_data es una cadena que necesita ser parseada
        elif isinstance(prediction_data, str):
            try:
                # Eliminar saltos de línea y espacios extra para ast.literal_eval
                cleaned_str = prediction_data.replace('\n ', '').strip()
                pred_list = ast.literal_eval(cleaned_str)
            except (ValueError, SyntaxError):
                # Si la cadena no es una lista de diccionarios válida, manejar el error
                # print(f"Advertencia: No se pudo parsear la cadena: '{prediction_data}'") # Descomentar para depurar
                return "UNKNOWN"
        # Caso 3: prediction_data es un numpy array u otro tipo inesperado
        elif isinstance(prediction_data, np.ndarray):
            # Si es un numpy array, intentamos convertirlo a lista y luego procesar
            try:
                pred_list = prediction_data.tolist()
            except AttributeError: # Si tolist() no está disponible
                # print(f"Advertencia: Tipo de datos inesperado para array: {type(prediction_data)}") # Descomentar para depurar
                return "UNKNOWN"
        else:
            # print(f"Advertencia: Tipo de datos inesperado para prediction: {type(prediction_data)}") # Descomentar para depurar
            return "UNKNOWN" # Retorna "UNKNOWN" para tipos de datos no manejados

        # Ahora que tenemos pred_list (o un fallo ya ocurrió)
        if pred_list and isinstance(pred_list, list):
            try:
                # Asegurarse de que cada elemento en pred_list es un diccionario con 'label' y 'score'
                if all(isinstance(x, dict) and 'label' in x and 'score' in x for x in pred_list):
                    # Encuentra el diccionario con el score más alto
                    dominant_pred = max(pred_list, key=lambda x: x['score'])
                    return dominant_pred['label']
            except (KeyError, TypeError):
                # print(f"Advertencia: Formato inesperado en elementos de la lista: {pred_list}") # Descomentar para depurar
                return "UNKNOWN"
        return "UNKNOWN" # Si pred_list está vacía o no es una lista válida

    df_cleaned['processed_label'] = df_cleaned[label_column_name].apply(get_dominant_label)


    # Función para crear el formato de mensaje de OpenAI con las columnas procesadas
    def create_openai_message(row):
        messages = [
            {"role": "user", "content": row['processed_text']},
            {"role": "assistant", "content": row['processed_label']}
        ]
        return {"messages": messages}

    # Aplicar la función a cada fila para crear la lista de entradas JSONL
    openai_data = df_cleaned.apply(create_openai_message, axis=1).tolist()

    # Definir el nombre del archivo JSONL de salida
    output_jsonl_file = 'banking_sentiment_finetune_final.jsonl' # Otro nombre para la versión más robusta

    # Guardar a JSONL
    with open(output_jsonl_file, 'w', encoding='utf-8') as f:
        for entry in openai_data:
            json.dump(entry, f, ensure_ascii=False)
            f.write('\n')

    print(f"Archivo '{output_jsonl_file}' generado exitosamente")
    print(f"Se procesaron {len(openai_data)} ejemplos.")
    print("\nLas primeras 3 entradas del archivo JSONL son:")
    with open(output_jsonl_file, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= 3:
                break
            print(line.strip())

except FileNotFoundError:
    print(f"Error: No se pudo encontrar el archivo parquet en la ruta: {parquet_file_path}")
except Exception as e:
    print(f"Ocurrió un error general al procesar el archivo: {e}")

Archivo 'banking_sentiment_finetune_final.jsonl' generado exitosamente
Se procesaron 5001 ejemplos.

Las primeras 3 entradas del archivo JSONL son:
{"messages": [{"role": "user", "content": "My card stopped working after multiple transactions. Why?"}, {"role": "assistant", "content": "NEGATIVE"}]}
{"messages": [{"role": "user", "content": "Hi, I made a transfer from France two days ago and thought it would be here by now. Can you give me an update please?"}, {"role": "assistant", "content": "POSITIVE"}]}
{"messages": [{"role": "user", "content": "Has my top-up been cancelled?"}, {"role": "assistant", "content": "NEGATIVE"}]}
